Anxiety is very common nowadays so in this part we try to build a model which predicts the level of anxiety based on the psychometric tests conducted without any doctoral supervision. We use the Generalised Anxiety Disorder(GAD) dataset based on proper official test questionnaire given to the subjects.



First we begin by loading the data and separating it into categorical and continuous features. Categorical feature are those which have classes or which are quantised whereas continuous features are not quantised as the name suggests. We convert the categorical features to integers using the labelencoder from sklearn. We create the target label ‘GroupedGAD’ which basically divides total GAD score into 4 categories 0-5, 5-10, 10-15, 15-22 which are the actual categories prescribed in the test and using which the doctors assess the patient’s anxiety level. This is what our model will finally predict.

For this task, we will use the TabTransformer model which is a very popular and widely used tabular model which has the power and prowess to take into account the tabular relations and the connections between various features. We implement this model from scratch so it can be a little difficult to follow in between but make sure to read the comments


In [ ]:
import pandas as pd
df = pd.read_csv('/content/GamingStudy_data.csv',encoding = 'ISO-8859-1')
print(df.shape)
df.head()

(13464, 55)


,S. No.,Timestamp,GAD1,GAD2,GAD3,GAD4,GAD5,GAD6,GAD7,GADE,...,Birthplace,Residence,Reference,Playstyle,accept,GAD_T,SWL_T,SPIN_T,Residence_ISO3,Birthplace_ISO3
0,1,42052.00437,0,0,0,0,1,0,0,Not difficult at all,...,USA,USA,Reddit,Singleplayer,Accept,1,23,5.0,USA,USA
1,2,42052.00680,1,2,2,2,0,1,0,Somewhat difficult,...,USA,USA,Reddit,Multiplayer - online - with strangers,Accept,8,16,33.0,USA,USA
2,3,42052.03860,0,2,2,0,0,3,1,Not difficult at all,...,Germany,Germany,Reddit,Singleplayer,Accept,8,17,31.0,DEU,DEU
3,4,42052.06804,0,0,0,0,0,0,0,Not difficult at all,...,USA,USA,Reddit,Multiplayer - online - with online acquaintanc...,Accept,0,17,11.0,USA,USA
4,5,42052.08948,2,1,2,2,2,3,2,Very difficult,...,USA,South Korea,Reddit,Multiplayer - online - with strangers,Accept,14,14,13.0,KOR,USA


In [ ]:
d = {range(0, 5): 0, range(5, 10): 1, range(10, 15): 2, range(15,22): 3}
df['groupedGAD'] = df['GAD_T'].apply(lambda x: next((v for k, v in d.items() if x in k), 0))

In [ ]:
# A list of the numerical feature names.
NUMERIC_FEATURE_NAMES = df.columns[df.dtypes=='int64'].tolist() + df.columns[df.dtypes=='float64'].tolist()

In [ ]:
NUMERIC_FEATURE_NAMES.remove('groupedGAD')
cat_cols = df.columns[df.dtypes=='object'].tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in df.columns[df.dtypes == object]:
    df[col] = df[col].fillna("NA")
    l_enc = LabelEncoder()
    df[col] = l_enc.fit_transform(df[col].values)

for col in df.columns[df.dtypes == 'float64']:
    df.fillna(df.loc[:, col].mean(), inplace=True)
for col in df.columns[df.dtypes == 'int64']:
    df.fillna(df.loc[:, col].min(), inplace=True)

In [ ]:
# A dictionary of the categorical features and their vocabulary.

CATEGORICAL_FEATURES_WITH_VOCABULARY = {}
for col in cat_cols:
  CATEGORICAL_FEATURES_WITH_VOCABULARY[col] = sorted(list(df[col].unique()))
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())
# A list of all the input features.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

# The name of the target feature.
TARGET_FEATURE_NAME = "groupedGAD"
# A list of the labels of the target features.
TARGET_LABELS = [0,1,2,3]

In [ ]:
df.shape

(13464, 56)

In [ ]:
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 265
NUM_EPOCHS = 15

NUM_TRANSFORMER_BLOCKS = 3  # Number of transformer blocks.
NUM_HEADS = 4  # Number of attention heads.
EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [
    2,
    1,
]  # MLP hidden layer units, as factors of the number of inputs.
NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

In [ ]:
!pip install --upgrade keras -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import keras
from keras import layers
from keras import ops
import math
import numpy as np
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="float32"
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="float32"
            )
    return inputs

In [ ]:
def encode_inputs(inputs, embedding_dims):
    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            embedding = layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(inputs[feature_name])
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:
            # Use the numerical features as-is.
            numerical_feature = ops.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list

In [ ]:
def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):
    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer()),
        mlp_layers.append(layers.Dense(units, activation=activation))
        mlp_layers.append(layers.Dropout(dropout_rate))

    return keras.Sequential(mlp_layers, name=name)

In [ ]:
def create_tabtransformer_classifier(
    num_transformer_blocks,
    num_heads,
    embedding_dims,
    mlp_hidden_units_factors,
    dropout_rate,
    use_column_embedding=False,
):
    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    print(encoded_categorical_feature_list)
    # Stack categorical feature embeddings for the Tansformer.
    encoded_categorical_features = ops.stack(encoded_categorical_feature_list, axis=1)
    # Concatenate numerical features.
    numerical_features = layers.concatenate(numerical_feature_list)

    # Add column embedding to categorical feature embeddings.
    if use_column_embedding:
        num_columns = encoded_categorical_features.shape[1]
        column_embedding = layers.Embedding(
            input_dim=num_columns, output_dim=embedding_dims
        )
        column_indices = ops.arange(start=0, stop=num_columns, step=1)
        encoded_categorical_features = encoded_categorical_features + column_embedding(
            column_indices
        )

    # Create multiple layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)
        # Skip connection 1.
        x = layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )
        # Layer normalization 1.
        x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)
        # Feedforward.
        feedforward_output = create_mlp(
            hidden_units=[embedding_dims],
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=partial(
                layers.LayerNormalization, epsilon=1e-6
            ),  # using partial to provide keyword arguments before initialization
            name=f"feedforward_{block_idx}",
        )(x)
        # Skip connection 2.
        x = layers.Add(name=f"skip_connection2_{block_idx}")([feedforward_output, x])
        # Layer normalization 2.
        encoded_categorical_features = layers.LayerNormalization(
            name=f"layer_norm2_{block_idx}", epsilon=1e-6
        )(x)

    # Flatten the "contextualized" embeddings of the categorical features.
    categorical_features = layers.Flatten()(encoded_categorical_features)
    # Apply layer normalization to the numerical features.
    numerical_features = layers.LayerNormalization(epsilon=1e-6)(numerical_features)
    # Prepare the input for the final MLP block.
    features = layers.concatenate([categorical_features, numerical_features])

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization,
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


tabtransformer_model = create_tabtransformer_classifier(
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    num_heads=NUM_HEADS,
    embedding_dims=EMBEDDING_DIMS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)


[<KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_39>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_40>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_41>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_42>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_43>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_44>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_45>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_46>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_47>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_48>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_49>, <KerasTensor shape=(None, 16), dtype=float32, sparse=False, name=keras_tensor_50>, <Ke

In [ ]:
def run_experiment(
    model,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
):
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
    )


    print("Start training the model...")
    history = model.fit(
        [df[col] for col in FEATURE_NAMES],df[TARGET_FEATURE_NAME], epochs=num_epochs
    )
    print("Model training finished")


    return history,model

In [ ]:
history,model = run_experiment(
    model=tabtransformer_model,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)

Start training the model...
Epoch 1/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 68s 73ms/step - accuracy: 0.6469 - loss: -2812.5935
Epoch 2/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6348 - loss: -67028.8125
Epoch 3/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6125 - loss: -317428.4062
Epoch 4/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.6108 - loss: -796476.6875
Epoch 5/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6091 - loss: -1560085.6250
Epoch 6/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6238 - loss: -2663391.5000
Epoch 7/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6028 - loss: -4303390.5000
Epoch 8/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6138 - loss: -5872069.5000
Epoch 9/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6250 - loss: -8231250.0000
Epoch 10/15
421/421 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6111 - loss: -10857761.0000
Epoch 11/15
421/421 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
preds = model.predict([df[col] for col in FEATURE_NAMES])

421/421 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


In [ ]:
print(df.iloc[0,:])
print()
print('Prediction: ', preds[0])

S. No.                 1.000000
Timestamp          42052.004370
GAD1                   0.000000
GAD2                   0.000000
GAD3                   0.000000
GAD4                   0.000000
GAD5                   1.000000
GAD6                   0.000000
GAD7                   0.000000
GADE                   2.000000
SWL1                   3.000000
SWL2                   5.000000
SWL3                   5.000000
SWL4                   5.000000
SWL5                   5.000000
Game                   8.000000
Platform               0.000000
Hours                 15.000000
earnings              62.000000
whyplay              279.000000
League               512.000000
highestleague      42054.841222
streams                0.000000
SPIN1                  1.000000
SPIN2                  0.000000
SPIN3                  0.000000
SPIN4                  1.000000
SPIN5                  0.000000
SPIN6                  0.000000
SPIN7                  1.000000
SPIN8                  0.000000
SPIN9   